In [14]:
from src.chat import Chat
from app.models.UserInput import UserInput

In [15]:
chat = Chat()


/Users/gcpuppy/Documents/WorkSpace/DataSci/dsde-project/src/database/qdrant.py:37: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv("combined_data.csv")


## Set System Prompt

In [16]:
user_message = "Suggests me some articles about watermelon"

chat_id = "abc"
user_input = UserInput(
    chat_id=chat_id,
    message=user_message,
    currentGraph=None
)

# Goal
1. Give "Keyword node"
2. Give "GrapLink from GraphData"

In [17]:
### FIX HERE
system_template_prompt = """You are an AI assistance for encouraging the system to initialize new "node" from below data that was fetched from QdrantDB. Node have two types which are "Paper" and "Keyword".
Keyword is a short word that related to topic, title or field of the input messages from user while "Paper" is the main content of articles in the database. Your tasks are generating "Keyword" node based on summarizing the content of user messages and "Paper" node, a node that contains more . Both of them follow this structure of node object: 

Don't forget to ensure that the values that will be assigned are related to data in the databas

This is the guideines for you:
1. Analyze the given user input to find some "Keyword" that related to the user message topic, papers, articles.
2. From the paper that user give to you, You have to find the top most important 4-5 topics and keep their id, title, type, year, abstract (if the abstract too long just keep as long as you think it's important and meaningful),  author and source in Json format.
3. After you have done the step 2, Create the nodes, this is the example structure. 
    "links" : [
        
            "source" : "2410.18541_arxiv",
            "target" : "2204.13154_arxiv"
        ,
        
            "source" : "transformer-and-attention-is-all-you-need",
            "target" : "2204.13154_arxiv"
        ,
        
            "source" : "transformer-and-attention-is-all-you-need",
            "target" : "2410.18541_arxiv"
        ,
    ]


Ensure that every pairs of the Graphlinks member are related and as you can see from the example, some graph link might have the same source, which store
the ID of the starter node (the node that have higher level).
Don't for get to change the id of source and target to the correct id of provided paper. The IDs in the examples isn't always the same as what users provide to you.
However, You can adjust the inhertance of the graph using your decision to make it be more appropriate.
4. Node with type "keyword" will have 3 fields [id], [type] and [title] (no label)
5. In the end, Return them in json format by Nodes and GraphLinks is in the same level and re-check if everything is correct and follows the guidelines, IDs in GraphLinks are paired with the IDs from provided papers.
6. Moreover, add one more section to the response Json, "Summation". It's a section to show the message that obtain from wraping the information up. You might add the title of the articles to make it be more easier to read. and don't for get to keep the conversation
in friendly mood and always encourage the user to ask you if they have any problems or need any helps.

"""
### FIX HERE

chat.set_system_template_prompt(system_template_prompt)

# Set Chat Template Prompt

In [18]:
### FIX HERE
chat_template_prompt = """
{paper_data}
"""
### FIX HERE

chat.set_chat_template_prompt(chat_template_prompt)

In [19]:
### See the result of the chat

initial_result = chat.test_initial_chat(user_input=user_input)

================================ Human Message =================================


{
    "id": "2303.16609_arxiv",
    "title": "Modified watershed approach for segmentation of complex optical\n  coherence tomographic images",
    "type": "paper",
    "year": 2023,
    "abstract": "  Watershed segmentation method has been used in various applications. But many\na times, due to its over-segmentation attributes, it underperforms in several\ntasks where noise is a dominant source. In this study, Optical Coherence\nTomography images have been acquired, and segmentation has been performed to\nanalyse the different regions of fluid filled sacs in a lemon. A modified\nwatershed algorithm has been proposed which gives promising results for\nsegmentation of internal lemon structures.\n",
    "authors": [
        "Maryam Viqar",
        " Violeta Madjarova",
        " Elena Stoykova"
    ],
    "source": "arxiv"
}
{
    "id": "85117283638.0_scopus",
    "title": "Comprehensive genome-wide analys

In [7]:
# chat.clear_chat(chat_id=chat_id)

In [20]:
chat.get_chat_history(chat_id=chat_id)

[HumanMessage(content='\n{\n    "id": "2303.16609_arxiv",\n    "title": "Modified watershed approach for segmentation of complex optical\\n  coherence tomographic images",\n    "type": "paper",\n    "year": 2023,\n    "abstract": "  Watershed segmentation method has been used in various applications. But many\\na times, due to its over-segmentation attributes, it underperforms in several\\ntasks where noise is a dominant source. In this study, Optical Coherence\\nTomography images have been acquired, and segmentation has been performed to\\nanalyse the different regions of fluid filled sacs in a lemon. A modified\\nwatershed algorithm has been proposed which gives promising results for\\nsegmentation of internal lemon structures.\\n",\n    "authors": [\n        "Maryam Viqar",\n        " Violeta Madjarova",\n        " Elena Stoykova"\n    ],\n    "source": "arxiv"\n}\n{\n    "id": "85117283638.0_scopus",\n    "title": "Comprehensive genome-wide analysis of calmodulin-binding transcript

In [21]:
from typing import List
from app.models.GraphData import GraphLink, Node, GraphData
import json

In [22]:
def format_chat_json_to_graph_link(chat_json_response: str) -> List[GraphLink]:
    """
    Example of chat_json_response
    ```json
    [
        {
            "source": "some_id",
            "target": "some_id",
            "index": 0
        }
    ]
    ```
    """

    # Select content between ```json and ```
    chat_json_response = chat_json_response.split("```json")[1].split("```")[0]
    # print(chat_json_response)
    graph_data = json.loads(chat_json_response)
    # print(graph_data)
    nodes_json = graph_data["nodes"]
    links_json = graph_data["links"]

    nodes = []
    for node in nodes_json:
        if "label" in node:
            node["title"] = node["label"]
            del node["label"]
        
        if "year" not in node:
            node["year"] = None
        
        if "abstract" not in node:
            node["abstract"] = None

        if "authors" not in node:
            node["authors"] = None

        if "source" not in node:
            node["source"] = None
            
        node = Node(**node)
        nodes.append(node)

    links = []
    for link in links_json:
        link = GraphLink(**link)
        links.append(link)

    return GraphData(nodes=nodes, links=links)

In [23]:
graph_data = format_chat_json_to_graph_link(initial_result.content)

# Set Detect Additional Data

In [24]:
detect_additional_message = "I want to know more about Computer"

In [25]:
user_detect_additional_data = UserInput(
    chat_id=chat_id,
    message=detect_additional_message,
    currentGraph=graph_data
)

In [26]:
detect_additional_data_template = """
These are your informative data for making the decision if user needs more papers or not:
- {message}
- This is the current graph of information that user has:
  {current_graph}

These are your guidelines :

Accordning to the amount, quality and usage of paper that related to the topic of the conversation. Do the user needs the more number of papers?
If there are some papers in current graph that related to the topic that user's interted in of researching.
If yes reply "Yes" Otherwise "No" followed by the reason of the answer. Furthermore, suggest the next appropriate step to the user by
considering the overall of this conversation detaily. Be calm, friendly, natural and love to give the best service to the user.
Return the response by this json template:
"isNeed" : "yes or no",
"reason" : "the reason that you suggest user to find more papers or the reason that the papaers is enough for user"

"""
chat.set_detect_additional_data_template(detect_additional_data_template)

In [27]:
detect_additional_data_result = chat.test_detect_additional_data(user_input=user_detect_additional_data)
detect_additional_data_result

================================== Ai Message ==================================

```json
{
  "isNeed": "yes",
  "reason": "The current papers and keywords are centered around fruit and image analysis, which doesn't align with your interest in computers. To give you relevant information, we need to explore papers related to computer science."
}
```
None


False

In [16]:
# def convert_detect_additional_data_to_boolean(detect_additional_data_response: str) -> bool:
#     return detect_additional_data_response.lower() == "yes"

In [17]:
# convert_detect_additional_data_to_boolean(detect_additional_data_result.content)

In [ ]:
continue_chat_result = chat.test_continue_chat(user_input=user_detect_additional_data)